# Code for NN with Artificial dataset

In [89]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

The function for generating datasets from Gaussian mixtures.

In [90]:
def gaussian_mix(m1,m2,m3,cov,n_samples):
    n_samples=int(n_samples/3)
    x, y = np.random.multivariate_normal(m1, cov, n_samples).T
    x=x.reshape((n_samples,1))
    y=y.reshape((n_samples,1))
    data1=np.hstack((x,y))
    x, y = np.random.multivariate_normal(m2, cov, n_samples).T
    x=x.reshape((n_samples,1))
    y=y.reshape((n_samples,1))
    data2=np.hstack((x,y))
    x, y = np.random.multivariate_normal(m3, cov, n_samples).T
    x=x.reshape((n_samples,1))
    y=y.reshape((n_samples,1))
    data3=np.hstack((x,y))
    data=np.vstack((data1,data2,data3))
    np.random.shuffle(data)
    return data

Generation of data from the central distribution.

In [91]:
n_samples=4000
n_samples/=4
x, y = np.random.multivariate_normal(m1, cov, n_samples).T
x=x.reshape((n_samples,1))
y=y.reshape((n_samples,1))
data1=np.hstack((x,y))
x, y = np.random.multivariate_normal(m2, cov, n_samples).T
x=x.reshape((n_samples,1))
y=y.reshape((n_samples,1))
data2=np.hstack((x,y))
x, y = np.random.multivariate_normal(m3, cov, n_samples).T
x=x.reshape((n_samples,1))
y=y.reshape((n_samples,1))
data3=np.hstack((x,y))
x, y = np.random.multivariate_normal(m4, cov, n_samples).T
x=x.reshape((n_samples,1))
y=y.reshape((n_samples,1))
data4=np.hstack((x,y))
data=np.vstack((data1,data2,data3,data4))
np.random.shuffle(data)
test=data
labels = np.sign(test[:,0]*test[:,1]).reshape((test.shape[0],1))

TypeError: 'float' object is unsliceable

Assigning labels to the central distribution.

In [92]:
lab=np.zeros((len(labels),2))
for i in range(len(labels)):
    if labels[i]==-1:
        lab[i][0]=1
    else:
        lab[i][1]=1

## Training of individual classifiers

In [93]:
m1=np.array([1,1])
m2=np.array([-1,1])
m3=np.array([-1,-1])
m4=np.array([1,-1])
cov=[[1,0],[0,1]]
X1 = gaussian_mix(m1,m2,m3,cov,5400)
y1 = np.sign(X1[:,0]*X1[:,1]).reshape((5400,1))

X2 = gaussian_mix(m1,m3,m4,cov,5400)
y2 = np.sign(X1[:,0]*X1[:,1]).reshape((5400,1))

print("Total samples in our dataset is: {}".format(X1.shape[0]))
n_samples = len(y1)

Total samples in our dataset is: 5400


In [94]:
val=np.zeros((len(y1),2))
for i in range(len(y1)):
    if y1[i]==-1:
        val[i][0]=1
    else:
        val[i][1]=1

In [95]:
x_train, x_test, y_train, y_test = train_test_split(X1, val, test_size=0.33, random_state=0)

In [96]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

def relu(X):
    return np.maximum(0,X)

def reluD(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [97]:
w0 = 2*np.random.random((2, 20)) - 1
w1 = 2*np.random.random((20, 2)) - 1
n=0.01
for i in range(100000):
    layer0 = x_train
    layer1 = relu(np.dot(layer0, w0))
    layer2 = sigmoid(np.dot(layer1, w1))
    layer2_error = y_train - layer2
    layer2_delta = layer2_error * sigmoid_deriv(layer2)
    layer1_error = layer2_delta.dot(w1.T)
    layer1_delta = layer1_error * reluD(layer1)
    w1 += layer1.T.dot(layer2_delta) * n
    w0 += layer0.T.dot(layer1_delta) * n
    error = np.mean(np.abs(layer2_error))
    accuracy = (1 - error) * 100
print("Training Accuracy " + str(round(accuracy,2)) + "%")

Training Accuracy 56.07%


In [98]:
layer0 = x_test
layer1 = relu(np.dot(layer0, w0))
layer2 = sigmoid(np.dot(layer1, w1))
layer2_error = y_test - layer2
error = np.mean(np.abs(layer2_error))
accuracy = (1 - error) * 100

print("Test Accuracy " + str(round(accuracy,2)) + "%")

Test Accuracy 57.38%


In [99]:
val2=np.zeros((len(y2),2))
for i in range(len(y2)):
    if y2[i]==-1:
        val2[i][0]=1
    else:
        val2[i][1]=1        
x_train, x_test, y_train, y_test = train_test_split(X2, val2, test_size=0.33, random_state=0)
w02 = 2*np.random.random((2, 20)) - 1
w12 = 2*np.random.random((20, 2)) - 1
n=0.01
for i in range(100000):
    layer02 = x_train
    layer12 = sigmoid(np.dot(layer02, w02))
    layer22 = sigmoid(np.dot(layer12, w12))
    layer22_error = y_train - layer22
    layer22_delta = layer22_error * sigmoid_deriv(layer22)    
    layer12_error = layer22_delta.dot(w12.T)
    layer12_delta = layer12_error * sigmoid_deriv(layer12)    
    w12 += layer12.T.dot(layer22_delta) * n
    w02 += layer02.T.dot(layer12_delta) * n    
    error = np.mean(np.abs(layer22_error))
    accuracy = (1 - error) * 100
print("Training Accuracy " + str(round(accuracy,2)) + "%")

Training Accuracy 56.57%


In [100]:
layer02 = x_test
layer12 = sigmoid(np.dot(layer02, w02))
layer22 = sigmoid(np.dot(layer12, w12))
layer22_error = y_test - layer22
error = np.mean(np.abs(layer22_error))
accuracy = (1 - error) * 100

print("Test Accuracy " + str(round(accuracy,2)) + "%")

Test Accuracy 44.27%


In [101]:
import math
def normpdf(x, mean):
    denom = (2*math.pi)
    num = math.exp(-np.dot((x-mean),(x-mean))/2)
    return num/denom

## Static mixing of hypothesis

In [102]:
normpdf(np.array([1,-1]),m1)

0.02153927930184863

In [114]:
layer0 = test
layer1 = relu(np.dot(layer0, w0))
layer2 = sigmoid(np.dot(layer1, w1))
layer12 = sigmoid(np.dot(layer0, w02))
layer22 = sigmoid(np.dot(layer12, w12))
pred=0.7*layer2+0.3*layer22
err1 = lab - layer2
err2 = lab - layer22
err = lab - pred
error = np.mean(np.abs(err))
accuracy = (1 - error) * 100

print("Test Accuracy: " + str(round(accuracy,2)) + "%")

error = np.mean(np.abs(err1))
accuracy = (1 - error) * 100
print("Test Accuracy with classifier 1: " + str(round(accuracy,2)) + "%")

error = np.mean(np.abs(err2))
accuracy = (1 - error) * 100
print("Test Accuracy with classifier 2: " + str(round(accuracy,2)) + "%")

Test Accuracy: 48.61%
Test Accuracy with classifier 1: 50.88%
Test Accuracy with classifier 2: 43.31%


## Dynamic mixing of hypothesis

In [115]:
layer0 = test
layer1 = relu(np.dot(layer0, w0))
layer2 = sigmoid(np.dot(layer1, w1))
layer12 = sigmoid(np.dot(layer0, w02))
layer22 = sigmoid(np.dot(layer12, w12))
pred = np.zeros(lab.shape)
l=np.zeros((labels.shape[0],1))
for i in range((len(l))):
    j=np.max(np.array([normpdf(test[i,:],m1),normpdf(test[i,:],m2),normpdf(test[i,:],m3)]))
    o=np.max(np.array([normpdf(test[i,:],m1),normpdf(test[i,:],m3),normpdf(test[i,:],m4)]))
    l[i]=j/(j+o)
    pred[i,:]=0.7*l[i]*layer2[i,:] + 0.3*(1-l[i])*layer22[i,:]
err = lab - pred
error = np.mean(np.abs(err))
accuracy = (1 - error) * 100
print("Test Accuracy: " + str(round(accuracy,2)) + "%")

Test Accuracy: 50.53%
